In [1]:
import sys
sys.path.append('../')

from model import *
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import pickle as pkl

from utils_synthetic import *

In [2]:
import sklearn
sklearn.__version__

'1.0.2'

# Load child welfare synthetic data

This notebook runs the analysis on the childwelfare data by leveraging experts' agreement
1. Explore a model build on data ignoring experts 
2. Compute agreement between experts using influence function
3. Retrain the model on the set of labels for which experts strongly agree

The current analysis uses multi layer perceptrons in a single train / test split.

### Data

Reopen the data created with the notebook in `data/`

In [3]:
data_file = '../../../data/semi_synthetic/Data_semisynthetic_v1.pkl'

In [4]:
with open(data_file, 'rb') as handle:
    X,Y_1,Y_2,Y,D_0,refer_ids,screener_ids,coef_pred_y = pkl.load(handle)

In [5]:
#drop instances if expert assessed a single case

drop_experts = []
for num in screener_ids:

    if screener_ids.count(num) < 10:

        drop_experts.append(num)

In [6]:
drop_idx = []
for index, elem in enumerate(screener_ids):
    if elem in drop_experts:
        drop_idx.append(index)

In [7]:
drop_idx

[53,
 59,
 63,
 64,
 65,
 1112,
 1150,
 1151,
 1154,
 3288,
 3289,
 9749,
 16779,
 16780,
 25172,
 38533]

In [8]:
Y.shape

(46544,)

In [9]:
X = np.delete(X,drop_idx,axis=0)
Y_1 = np.delete(Y_1,drop_idx,axis=0)
Y_2 = np.delete(Y_2,drop_idx,axis=0)
Y = np.delete(Y,drop_idx,axis=0)
D_0 = np.delete(D_0,drop_idx,axis=0)
refer_ids = np.delete(refer_ids,drop_idx,axis=0)
screener_ids = np.delete(screener_ids,drop_idx,axis=0)

In [10]:
selective_labels = True
#noise = True
opb = True
opb_blind = False

unobservables = False
unobs_k = 5 #number of unobsevables, k features with largest coefficient

change_some_coef = False #resample some coefficients for each human?
change_same = False
n=44#how many coefficients to change if change_some_coef == True
shared_bias = False
bias_opposite = False #if shared_bias true, should the bias overestimate the importance of use it in the opposite direction?

bias_assignment = False
change_all_coef = False#resample all non-zero coefficients?

random_if_not_good = False

#If opb_out, modeled as a business rule?
business_rule = False

#selective labels? Do we only observe label when D=1?


#HUMAN DECISIONS MODEL PARAMETERS

rand = False #are decisions made by humans random?

In [11]:
if not opb:
        Y = Y_1
elif opb_blind and not business_rule:
    Y = np.array([((Y_1[i]==1)&(D_0[i]==0)) for i in np.arange(len(Y_1))])
    Y_2 = 1-D_0
    logit = linear_model.LogisticRegression(penalty = 'l1', C=0.01, random_state=42, fit_intercept=False)
    clf = logit.fit(X, Y)
    Y_pred = clf.predict_proba(X)
    fpr, tpr,thres = sklearn.metrics.roc_curve(Y, Y_pred[:,1])
    roc_auc = sklearn.metrics.auc(fpr, tpr)
    #print(roc_auc)
    coef_pred_y = clf.coef_
    #print(sum(coef_pred_y[0]!=0))
    #plt.plot(fpr, tpr, color='darkorange',label='ROC curve (area = %0.2f)' % roc_auc)
elif opb_blind and business_rule:
    Y_2 = 1-D_0



if unobservables: #delete one of the variables that receive a lot of weight
    X_obs = np.delete(X,np.argsort(coef_pred_y[0])[-(unobs_k+2):-2],1)
else:
    X_obs = X

if bias_assignment:
    screener_ids = np.array(screener_ids)
    screener_ids[X[:,-2] == max(X[:,-2])] = 'TNew'
    screener_ids=list(screener_ids)

screener_set = np.array([x for x in set(screener_ids) if str(x)!='nan'])

D, alphas = decision_model(X, screener_ids, screener_set, coef_pred_y[0], change_coef = change_some_coef, change_same = change_same, change_all=change_all_coef, n=n,  shared_bias=shared_bias, rand= rand, bias_opposite=bias_opposite, bias_assignment= bias_assignment, random_if_not_good = random_if_not_good)

if opb and opb_blind and business_rule:
    D[D_0] = 0
    Y[D_0] = 0

#     with open('../../data/semi_synthetic/Y_human_'+setting+'.pkl', 'wb') as file:
#         pkl.dump([X,Y_1,Y_2,Y,D_0,refer_ids,screener_ids,coef_pred_y,D],file)
print(sum(D)/len(D))   
print(sum(D==Y)/len(D)   )

0.3371662316602846
0.15438842991404697
1.2789898467937826
-0.5668497903127191
-0.1829263795552658
-0.062173478366350274
0.16686433336210982
-0.5196630298510732
0.8557966681575077
-0.41254153918521785
-0.034568934494623435
-0.2999388973192902
-0.26663512234945325
0.15126214282745998
0.2668092773520606
0.23022723616032145
0.7164520061564308
0.34504177535971065
0.2520944370709525
-0.03751832852234079
0.16326508177332555
0.9086757588320169
1.062757900298296
0.34004277245411335
-0.4127702969163972
0.4762979117261796
0.3451015770506714
-0.541994235684026
-0.8851478283605279
-0.7422020795708525
0.042318364308891826
1.0994399316965144
-0.027574458210725825
0.453296939477304
0.453296939477304
0.8813402682255846


In [12]:
Y_2 = Y_2*1

In [13]:
YC = [max(Y_1[i], Y_2[i]) for i in np.arange(len(Y_1))]

In [14]:
target = pd.DataFrame({'D': D, 'Y1': Y_1, 'Y2': Y_2, 'YC': YC})

In [15]:
target

,D,Y1,Y2,YC
0,1.0,1.0,1,1.0
1,1.0,1.0,1,1.0
2,1.0,0.0,1,1.0
3,0.0,1.0,0,1.0
4,0.0,0.0,0,0.0
...,...,...,...,...
46523,1.0,0.0,1,1.0
46524,0.0,0.0,0,0.0
46525,0.0,0.0,0,0.0
46526,0.0,0.0,0,0.0


In [16]:

#covariates, target, nurses = triage.drop(columns = ['D', 'Y1', 'Y2', 'YC', 'acuity', 'nurse']), triage[['D', 'Y1', 'Y2', 'YC']], triage['nurse']

In [17]:
# ids_map = {}
# screener_ids
# for i in range(len(set(screener_ids))):
#     ids_map[]

In [18]:
#convert screener ids to integers
#screener_ids = [int(i[2:]) for i in screener_ids]

Split data in a 80% train, 20% test

In [19]:
cov_train, cov_test, tar_train, tar_test, nur_train, nur_test = train_test_split(pd.DataFrame(X), target, pd.Series(screener_ids), test_size = 0.2, random_state = 42)

### Modelling

In [20]:
# Model's characteristics
params = {'layers': []} # If = [] equivalent to a simple logistic regression

# Amalgation parameters
rho = 0.05 # Control which point to consider from a confience point of view
pi_1 = 4.0 # Control criterion on centre mass metric
pi_2 = 0.8 # Control criterion on opposing metric
tau = 1.0  # Balance between observed and expert labels

##### 1. Train on decision

This model models the nurse decision based on covariates

In [21]:
for l1_penalty in [0.001, 0.01]:
    try:
        model = BinaryMLP(**params)
        model = model.fit(cov_train, tar_train['D'], nur_train, l1_penalty = l1_penalty)
        break
    except Exception as e:
        print(e, l1_penalty)
        pass

Loss: 0.323: 100%|██████████| 1/1 [00:00<00:00,  2.81it/s]


In [22]:
# Naive performance
roc_auc_score(tar_test['Y1'], model.predict(cov_test))

0.9197947450603479

In [23]:
# Yc performance
roc_auc_score(tar_test['Y2'], model.predict(cov_test))

0.8606437059723931

In [24]:
roc_auc_score(tar_test['YC'], model.predict(cov_test))

0.949400611791916

In [25]:
roc_auc_score(tar_test['D'], model.predict(cov_test))

0.9571604173914081

##### 2. Agreement computation 

Measure of agreeability are estimated in a cross validation fashion on the train set.

In [26]:
from collections import Counter
Counter(screener_ids)

Counter({'T095467': 4119,
         'T096208': 4973,
         'T060875': 2483,
         'T084791': 3499,
         'T091442': 2400,
         'T094595': 2384,
         'T044996': 1642,
         'T066265': 4137,
         'T071543': 142,
         'T096966': 2834,
         'T098306': 1245,
         'T055473': 4877,
         'T079344': 4547,
         'T069459': 642,
         'T092753': 1123,
         'T098337': 701,
         'T078534': 27,
         'T087566': 382,
         'T102928': 444,
         'T102948': 59,
         'T098938': 33,
         'T098007': 161,
         'T101142': 2135,
         'T104299': 196,
         'T083062': 405,
         'T101854': 181,
         'T105008': 43,
         'T104369': 77,
         'T105222': 42,
         'T105208': 30,
         'T099724': 44,
         'T099665': 495,
         'T105837': 26})

In [49]:
screener_ids.dtype

dtype('<U7')

In [60]:
def influence_cv(model, x, y, h, params = {}, fit_params = {}, n_split = 3):
    """
    Compute a stratified cross validation to estimate the influence of each points

    Args:
        model (Object): Create a model (need to have predict and influence functions)
        x (np.array pd.DataFrame): Covariates
        y (np.array pd.DataFrame): Associated outcome
        h (np.array pd.DataFrame): Associated expert
        params (Dict): Dictionary to initialize the model with
        fit_params (Dict): Dictionary for training
        split (int): Number of fold used for the stratified computation of influence

    Returns:
        folds, predictions, influence: Arrays of each point fold, predictions by the model and influence (dim len(x) * num experts)
    """
    x, y, h = (x.values, y.values, h.values) if isinstance(x, pd.DataFrame) else (x, y, h)

    # Shuffle data - Need separation from fold to ensure group
    sort = np.arange(len(h))
    np.random.seed(42)
    np.random.shuffle(sort)
    x, y, h = x[sort], y[sort], h[sort]
    print(h)
    # Create groups of observations to ensure one expert in each fold
    #h_unique = np.unique(h)
    g, unique_h = np.zeros_like(h), np.unique(h)
    for expert in unique_h:
        selection = h == expert
        g[selection] = np.arange(np.sum(selection))

    splitter = StratifiedGroupKFold(n_split, shuffle = False)
    folds, predictions, influence = np.zeros(len(x)), np.zeros(len(x)), np.zeros((len(unique_h), x.shape[0]))
    for i, (train_index, test_index) in enumerate(splitter.split(x, y, g)):
        folds[test_index] = i
        train_index, val_index = train_test_split(np.array(train_index), test_size = 0.15, shuffle = False)

        # Train model on the subset
        model_cv = model(**params)
        model_cv.fit(x[train_index], y[train_index], h[train_index], **fit_params, val = (x[val_index], y[val_index]))

        # Calibrate NN on validation set - Platt
        pred_val = model_cv.predict(x[val_index])
        pred_test = model_cv.predict(x[test_index])
        calibrated = LogisticRegression().fit(pred_val, y[val_index])
        predictions[test_index] = calibrated.predict_proba(pred_test)[:, 1]

        # Compute influence
        influence[:, test_index] = model_cv.influence(x[test_index])

    return folds, predictions, influence

In [61]:
# Fold evaluation of influences
folds, predictions, influence = influence_cv(BinaryMLP, cov_train, tar_train['D'], nur_train, params = params)

['T101142' 'T044996' 'T055473' ... 'T095467' 'T044996' 'T096966']
T044996
1324
T055473
3858
T060875
1979
T066265
3347
T069459
527
T071543
119
T078534
24
T079344
3637
T083062
313
T084791
2805
T087566
307
T091442
1916
T092753
881
T094595
1921
T095467
3325
T096208
3970
T096966
2273
T098007
129
T098306
996
T098337
558
T098938
26
T099665
401
T099724
33
T101142
1675
T101854
142
T102928
351
T102948
45
T104299
168
T104369
61
T105008
34
T105208
25
T105222
31
T105837
21


Loss: 0.362: 100%|██████████| 1/1 [00:00<00:00,  4.27it/s]


In [ ]:
# Compute metrics agreeability
center_metric, opposing_metric = compute_agreeability(influence)

In [10]:
# Apply criteria on amalgamation
high_conf = (predictions > (1 - rho)) | (predictions < rho)
high_agr = (center_metric > pi_1) & (opposing_metric > pi_2) & high_conf
high_agr_correct = ((predictions - tar_train['D']).abs() < rho) & high_agr

In [11]:
# Create amalgamated labels
tar_train['Ya'] = tar_train['Y1'].copy()
tar_train['Ya'][high_agr_correct] = (1 - tau) * tar_train['Y1'][high_agr_correct] \
                                    + tau * tar_train['D'][high_agr_correct]

/tmp/ipykernel_13443/148226879.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tar_train['Ya'] = tar_train['Y1'].copy()
/home/vincent/miniconda3/envs/Jupyter/lib/python3.9/site-packages/pandas/core/generic.py:8870: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [12]:
index_amalg = tar_train['D'] | high_agr_correct

##### 3. Updated model

In [20]:
model = BinaryMLP(**params)
model = model.fit(cov_train[index_amalg], tar_train[index_amalg]['Ya'], nur_train[index_amalg])

Loss: 0.623:   2%|▏         | 23/1000 [00:08<06:13,  2.62it/s]


In [21]:
# Naive performance
roc_auc_score(tar_test['Y1'], model.predict(cov_test))

0.6490108254809583

In [22]:
# Yc performance
roc_auc_score(tar_test['YC'],model.predict(cov_test))

0.5658628634431628

##### 4. Train on observed data

In [16]:
model = BinaryMLP(**params)
model = model.fit(cov_train, tar_train['Y1'], nur_train)

Loss: 0.687:  13%|█▎        | 128/1000 [01:55<13:04,  1.11it/s]


In [17]:
# Naive performance
roc_auc_score(tar_test['Y1'], model.predict(cov_test))

0.6532088012868359

In [18]:
# Yc performance
roc_auc_score(tar_test['YC'],model.predict(cov_test))

0.5464182542438537